In [52]:
%load_ext autoreload
%autoreload 2
import os
from pathlib import Path
from natsort import natsorted
import pandas as pd
import zarr
import sys
from skimage.feature import peak_local_max
from skimage.filters import gaussian
import napari
src_dir = os.environ.get('SRC_DIR')
if src_dir and src_dir not in sys.path:
    sys.path.append(src_dir)
from data import get_experiment_paths
data_dir = Path(os.environ.get('DATA_DIR'))
paths = get_experiment_paths(data_dir, "TS_5_4")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
arr = zarr.open(data_dir / paths["images"]["denoised"])[0]


In [9]:
v = napari.Viewer()

In [10]:
# v.add_image(arr)
v.add_image(gaussian_filtered)

<Image layer 'gaussian_filtered' at 0x33cc5bee0>

In [28]:
coordinates_all = []
for sigma in [5, 10, 15]:
    gaussian_filtered = gaussian(arr, sigma)
    coordinates = peak_local_max(1-gaussian_filtered, min_distance=sigma)
    v.add_image(gaussian_filtered)
    v.add_points(coordinates)
    coordinates_all.append(coordinates)

In [29]:
import numpy as np
coordinates_all = np.vstack(coordinates_all)

In [22]:
from sklearn.neighbors import NearestNeighbors

In [23]:
particles = pd.read_csv(data_dir / "objects.csv")

In [26]:
particles = particles.query("experiment == 'TS_5_4'")[["x", "y", "z"]]

In [30]:
particles, coordinates_all

(            x         y         z
 0    4601.271   601.066   600.934
 1    4803.789   455.425   514.016
 2    4715.436   825.374   802.166
 3    5003.275   782.745   802.480
 4     710.459  3815.845  1405.435
 ..        ...       ...       ...
 135  2175.491  3100.549   452.861
 136  2335.068  4957.432   703.716
 137  2752.287  5760.574   881.249
 138  2613.583  2253.208   884.042
 139   534.129  1666.452  1038.000
 
 [140 rows x 3 columns],
 array([[ 70, 384, 617],
        [ 24, 223, 620],
        [100, 435, 613],
        ...,
        [ 87, 265, 164],
        [ 76,  62,  26],
        [137, 521,  45]]))

In [48]:
particles.mean()

x    2939.361214
y    3137.947057
z     717.194236
dtype: float64

In [51]:
(coordinates_all*10)[:,::-1].mean(axis=0)

array([3048.93828452, 3025.08368201,  972.31433054])

In [44]:
distances = NearestNeighbors(n_neighbors=2).fit((coordinates_all*10)[:,::-1]).kneighbors(particles.values)[0][:, 0]

In [46]:
len(distances)

140